In [ ]:
# Import necessary libraries for the agent
from strands import Agent
from strands.models.openai import OpenAIModel  # For using OpenAI models
from strands.tools import tool  # For creating tools the agent can use
from langchain_community.vectorstores.faiss import FAISS


In [3]:
import os
model = OpenAIModel(
    client_args={
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
    # **model_config
    model_id="gpt-4o",
    params={
        "max_tokens": 1000,
        "temperature": 0.7,
    }
)


In [6]:
from langchain_openai import OpenAIEmbeddings  # or whatever embedding model you used

# Setup embedding model — same as when you built FAISS
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# Persist directory where your FAISS index is saved
PERSIST_DIR = "vectordb_openai/"  # change if your path is different

# Define a Strands tool for retrieval
@tool
def aws_course_search(query: str, k: int = 5) -> str:
    """
    Search AWS course PDF content (embedded in FAISS) for relevant text.

    Args:
        query (str): The user’s search question.
        k (int): Number of top similar chunks to retrieve.

    Returns:
        str: Joined text of the top-k relevant PDF chunks.
    """
    # Load the FAISS vector store (with your embedding model)
    vs = FAISS.load_local(PERSIST_DIR, embedding_model, allow_dangerous_deserialization=True)
    
    # Perform similarity search
    docs = vs.similarity_search(query, k=k)
    
    # Format the retrieved documents into a string
    if not docs:
        return "No relevant information found in the AWS course material."
    
    output = []
    for i, doc in enumerate(docs):
        # doc.metadata may have 'source', page, or any metadata you stored
        src = doc.metadata.get("source", "unknown")
        page = doc.metadata.get("page", None)
        prefix = f"Chunk {i+1} (Source: {src}"
        if page is not None:
            prefix += f", Page: {page}"
        prefix += "):\n"
        
        output.append(prefix + doc.page_content)
    
    return "\n\n".join(output)



In [7]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

In [8]:
@tool
def current_time(timezone: str = None) -> str:
    """
    Get the current time in ISO 8601 format.

    This tool returns the current date and time in ISO 8601 format (e.g., 2023-04-15T14:32:16.123456+00:00)
    for the specified timezone. If no timezone is provided, the value from the DEFAULT_TIMEZONE
    environment variable is used (defaults to 'UTC' if not set).

    Args:
        timezone (str, optional): The timezone to use (e.g., 'UTC', 'US/Pacific', 'Europe/London', 'Asia/Tokyo').
            Defaults to environment variable DEFAULT_TIMEZONE ('UTC' if not set).

    Returns:
        str: The current time in ISO 8601 format.

    Raises:
        ValueError: If an invalid timezone is provided.

    Examples:
        >>> current_time()  # Returns current time in default timezone (from DEFAULT_TIMEZONE or UTC)
        '2023-04-15T14:32:16.123456+00:00'

        >>> current_time(timezone="US/Pacific")  # Returns current time in Pacific timezone
        '2023-04-15T07:32:16.123456-07:00'
    """
    # Get environment variables at runtime
    default_timezone = os.getenv("DEFAULT_TIMEZONE", "UTC")

    # Use provided timezone or fall back to default
    timezone = timezone or default_timezone

    try:
        if timezone.upper() == "UTC":
            timezone_obj: Any = tz.utc
        else:
            timezone_obj = ZoneInfo(timezone)

        return datetime.now(timezone_obj).isoformat()
    except Exception as e:
        raise ValueError(f"Error getting current time: {str(e)}") from e

In [11]:
# Create a Strands Agent with this tool
import uuid  # For generating unique IDs

agent = Agent(
    name="AWS Course Assistant",
    model=model,
    tools=[aws_course_search, current_time],
    system_prompt=(
        "You are an assistant knowledgeable about the AWS course PDF content. "
        "For any question related to the course, use the aws_course_search tool to look up relevant snippets."
    ),
    record_direct_tool_call = True,  # Record when tools are used
    trace_attributes={
        "session.id": str(uuid.uuid4()),  # Generate a unique session ID
        "user.id": "user-email-example@domain.com",  # Example user ID
        "langfuse.tags": [
            "Agent-SDK-Example",
            "Strands-Project-Demo",
            "Observability-Tutorial"
        ]
    }
)





In [12]:
# 5. Use the agent
if __name__ == "__main__":
    while True:
        user_query = input("Ask about the AWS course: ")
        resp = agent(user_query)
        print("Agent:", resp)


Tool #1: aws_course_search
Amazon SageMaker is a fully managed service designed for developers and data scientists to build machine learning (ML) models. It simplifies the typically complex process of building, training, and deploying ML models by providing an end-to-end solution. Here are some key aspects of Amazon SageMaker:

- **End-to-End ML Service**: SageMaker allows you to collect and prepare data, build and train machine learning models, and deploy these models while monitoring the performance of predictions.
  
- **Model Deployment & Inference**: It provides one-click deployment with automatic scaling and no servers to manage, which reduces overhead. It supports real-time predictions, serverless architecture, and can handle idle periods between traffic spikes, although it may tolerate more latency due to cold starts.

SageMaker JumpStart, part of SageMaker, offers pre-built solutions and models to help accelerate the development process for common use cases.Agent: Amazon Sage

KeyboardInterrupt: Interrupted by user